In [1]:
sc

Intitializing Scala interpreter ...

Intitializing Scala interpreter ...

Spark Web UI available at http://60d52afe8fec:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591436133725)
SparkSession available as 'spark'


Spark Web UI available at http://60d52afe8fec:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591436133725)
SparkSession available as 'spark'


2020-06-06 09:35:31,651 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-06-06 09:35:31,651 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@6686b904


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@6686b904


In [2]:
//Start a simple Spark Session
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._

//Feature Processing Classes
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder, PCA}

//standard scaler
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.StandardScaler

//Linear Algebra Data Structures
import org.apache.spark.ml.linalg.{Vector,Vectors}

//Model Building Pipeline
import org.apache.spark.ml.{Pipeline, PipelineStage, PipelineModel}

//Binary Classification
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel,
                                           RandomForestClassifier, GBTClassifier,
                                           DecisionTreeClassifier, DecisionTreeClassificationModel}
//Model Training
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.{CrossValidator, CrossValidatorModel, 
                                   ParamGridBuilder, TrainValidationSplit}

//Model Evaluation
import org.apache.spark.ml.evaluation.{BinaryClassificationEvaluator,MulticlassClassificationEvaluator}

//Optional: Use the following code below to set the Error reporting
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)


//For Cleaning
//import scala.util.matching.Regex

val spark = SparkSession.builder().appName("Group 8 ML Phase 3").getOrCreate()

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder, PCA}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.{Pipeline, PipelineStage, PipelineModel}
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel, RandomForestClassifier, GBTClassifier, DecisionTreeClassifier, DecisionTreeClassificationModel}
import org.apache.spark.ml.param.ParamMap
import org.apache.s...

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder, PCA}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.{Pipeline, PipelineStage, PipelineModel}
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel, RandomForestClassifier, GBTClassifier, DecisionTreeClassifier, DecisionTreeClassificationModel}
import org.apache.spark.ml.param.ParamMap
import org.apache.s...

In [3]:
// Take a random sample (without replacement) of the data (to reduce memory requirements)
val sampleFraction = 0.2

val flights = (spark
            .read.parquet("flightDelay.parquet")
            .withColumn("Month_Num1", $"Month_Num" cast "Int")
            //convert month and year to integer index starting Jan 2004
            .withColumn("Date_Num",  ($"Year"-2004)*12 + $"Month_Num1")
            .drop("Sectors_Flown", "Month_Num1", "Change")
            .withColumnRenamed("Departures_Delayed","label")
            .withColumnRenamed("Price","Fuel_Price")
            //take a sample without replacement
            .sample(false,sampleFraction, seed = 222)
            //drop NA's even though none were found!
            .na.drop()
            //.cache
              )

flights.printSchema()

root
 |-- Departing_Port: string (nullable = true)
 |-- Arriving_Port: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month_Num: string (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- Departing_Port_station_ID: string (nullable = true)
 |-- Departing_Port_station_name: string (nullable = true)
 |-- Arriving_Port_station_ID: string (nullable = true)
 |-- Arriving_Port_station_name: string (nullable = true)
 |-- Mean_3pm_cloud_cover_oktas_Depart: double (nullable = true)
 |-- Mean_3pm_dew_point_temperature_Degrees_C_Depart: double (nullable = true)
 |-- Mean_3pm_relative_humidity_%_Depart: double (nullable = true)
 |-- Mean_3pm_temperature_Degrees_C_Depart: double (nullable = true)
 |-- Mean_3pm_wet_bulb_temperature_Degrees_C_Depart: double (nullable = true)
 |-- Mean_3pm_wind_speed_km/h_Depart: double (nullable = true)
 |-- Mean_9am_cloud_cover_okas_Depart: double (nullable 

sampleFraction: Double = 0.2
flights: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 72 more fields]


sampleFraction: Double = 0.2
flights: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 72 more fields]


In [4]:
val counts = flights.groupBy("label").count()

println("proportion of lates (label=1) in the sample")
counts.show()

proportion of lates (label=1) in the sampleproportion of lates (label=1) in the sample

+-----+-------+
|label|  count|
+-----+-------+
|    1| 213092|
|    0|1045051|
+-----+-------+
+-----+-------+
|label|  count|
+-----+-------+
|    1| 213092|
|    0|1045051|
+-----+-------+




counts: org.apache.spark.sql.DataFrame = [label: int, count: bigint]


counts: org.apache.spark.sql.DataFrame = [label: int, count: bigint]


In [ ]:
flights.select("Date_Num").show(5)

### Creating Balanced training and test set

In [ ]:
//Filter out the most recent 12 months of flight data as the test dataset
//Dates after March 2019 have Date_Num > 183
val testing = flights.filter($"Date_Num"> 183)
println(s"Test Set of the Most Recent 12 Months has ${testing.count()} records")

//Filter out rows prior to the most recent 12 months of flight data as the training dataset
val rawTraining = flights.filter($"Date_Num" < 184)

//val Array(imbalancedTraining, testing) = flights.randomSplit(Array(0.7, 0.3), seed = 12345)

In [ ]:
val ontimeTestFlights = testing.filter($"label" === 0)
println(s"On time Test Flights: ${ontimeTestFlights.count()}")

val delayedTestFlights = testing.filter($"label" === 1)
println(s"Delayed Test FLights: ${delayedTestFlights.count()}")

val sampledOntimeTestFlights = ontimeTestFlights.sample(false, 0.2)

val sampleFraction = 0.1

val test = (sampledOntimeTestFlights
                .union(delayedTestFlights)
                .sample(false, sampleFraction))

//val sampledCounts = test.groupBy("label").count()
//println("proportion of lates (label=1) in the sample")
//sampledCounts.show()
               


In [ ]:
val ontimeTrainingFlights = rawTraining.filter($"label"===0)
println(s"On time Training Flights: ${ontimeTrainingFlights.count()}")

val delayedTrainingFlights = rawTraining.filter($"label"===1)
println(s"Delayed Training Flights: ${delayedTrainingFlights.count()}")

val sampledOntimeTrainingFlights = ontimeTrainingFlights.sample(false, 0.2)  

println(s"Down Sampled ontime Training Flights: ${sampledOntimeTrainingFlights.count()}")

val sampleFraction = 0.1
//Concatenate rows of ontimeTrainingFlights and delayedTrainingFlights
val training = (sampledOntimeTrainingFlights
                .union(delayedTrainingFlights)
                .sample(false, sampleFraction))
               
//val resampledCounts = training.groupBy("label").count()
//println("proportion of lates (label=1) in the sample")
//resampledCounts.show()
//training.count()

In [ ]:
//training.count()

### Confusion Matrix

In [ ]:
def getConfusionMatrix(predictionDF: DataFrame): Unit = {
    val eval = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction")
    println(s"Accuracy: ${eval.setMetricName("accuracy").evaluate(predictionDF)}")
    println(s"Weighted Precision: ${eval.setMetricName("weightedPrecision").evaluate(predictionDF)}")
    println(s"Weighted Recall: ${eval.setMetricName("weightedRecall").evaluate(predictionDF)}")
    println(s"F1: ${eval.setMetricName("f1").evaluate(predictionDF)}")

    val TP = predictionDF.select("label", "prediction").filter("label = 1 and prediction = 1").count
    val TN = predictionDF.select("label", "prediction").filter("label = 0 and prediction = 0").count
    val FP = predictionDF.select("label", "prediction").filter("label = 0 and prediction = 1").count
    val FN = predictionDF.select("label", "prediction").filter("label = 1 and prediction = 0").count
    val total = predictionDF.select("label").count.toDouble
    // Unweighted Metrics
    val accuracy    = (TP + TN) / total
    val precision   = TP / (TP + FP)
    val recall      = TP / (TP + FN)
    val F1 = 2/(1/precision + 1/recall)
    println(s"Accuracy: ${accuracy}")
    println(s"Precision: ${precision}")
    println(s"Recall: ${recall}")
    println(s"F1: ${F1}")

    // Confusion matrix
    printf(s"""|=================== Confusion Matrix ==========================
           |##########| %-15s                     %-15s
           |----------+----------------------------------------------------
           |Actual = 0| %-15d                     %-15d
           |Actual = 1| %-15d                     %-15d
           |===============================================================
         """.stripMargin, "Predicted = 0", "Predicted = 1", TN, FP, FN, TP)


}

### ML Pipeline

### preprocessing for ML

### Step 1  Extracting Weather feature columns along with Date_Num

In [ ]:
// using vector assembler to create a vector column of weather data


val array = Array("Departing_Port", "Arriving_Port", "Airline", "label", "Year", "Month_Num", "Fuel_Price", 
                  "Departing_Port_station_ID", "Departing_Port_station_name", "Arriving_Port_station_ID", 
                  "Arriving_Port_station_name")
val columns = flights.columns

val feature_cols = columns.filter(!array.contains(_))

//for(feature <- feature_cols) { println(feature) }

//println(s"length of feature_cols is ${feature_cols.length}")


### 1.1  Assembling weather data

In [ ]:
val weather_assembler = new VectorAssembler()
                        .setInputCols(feature_cols)
                        .setOutputCol("weather_features")

val weathered_DF = weather_assembler.transform(flights)

//weathered_DF.printSchema
val weather_features_df = weathered_DF.select("weather_features")
weather_features_df.show(5)

###  1.2 Before using PCA

Before we can reduce the dimensionality of the dataset using PCA, we first need to standardize the features that we described previously. This can be achieved using MLlib's StandardScaler estimator and fitting it to the Spark dataframe containing our feature vectors

In [ ]:
import org.apache.spark.ml.feature.StandardScaler

val std_scalar = new StandardScaler() 
                .setInputCol("weather_features")
                .setOutputCol("standard")
                .setWithStd(true)
                .setWithMean(false)


In [ ]:
//val standard_model = std_scalar.fit(weather_features_df)

In [ ]:
//val standard_weather_features_DF = standard_model.transform(weather_features_df)

In [ ]:
//standard_weather_features_DF.show(5)

### 1.3

In [ ]:
//val features_rdd = standard_weather_features_DF.select("standard").rdd

In [ ]:
//features_rdd.take(5).foreach(x=> println(x))

val pca = new PCA()
          .setInputCol("standard")
          .setOutputCol("continuos_features")
          .setK(14) // depart and arrive weather data

In [ ]:
// val features_matrix = RowMatrix(features_rdd.map(lambda x: x[0].tolist()))

### Step 2.  pre processing categorical features

In [ ]:
// Deal with Categorical Columns
val categoricalVariables = Array(
    "Departing_Port", "Arriving_Port", "Airline")
val categoricalIndexers = categoricalVariables
  .map(i => new StringIndexer().setInputCol(i).setOutputCol(i+"_Index"))
val categoricalEncoders = categoricalVariables
  .map(e => new OneHotEncoder().setInputCol(e + "_Index").setOutputCol(e + "_Vec"))

In [ ]:
// select the flight data explanatory categorica fields only that will predict flight delay
val explanatoryFields = Array("Airline_Vec", "Fuel_Price","Departing_Port_Vec","Arriving_Port_Vec")

// Assemble everything together to be ("label","features") format
val categorical_assembler = new VectorAssembler()
                 .setInputCols(explanatoryFields)
                 //.setOutputCol("indexedFeatures")
                 .setOutputCol("categorical_features")


In [ ]:
// creating features

val assembler = new VectorAssembler()
                .setInputCols(Array("continuos_features", "categorical_features"))
                .setOutputCol("features")

In [ ]:
val featureProcessingStages: Array[PipelineStage] = Array(weather_assembler, std_scalar, pca) ++ categoricalIndexers ++ 
                             categoricalEncoders ++ Array(categorical_assembler) ++ Array(assembler)

### Estimator component

In [ ]:
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.001)
  .setFeaturesCol("features")

### pipeline

In [ ]:
val pipeline = new Pipeline()
  .setStages(featureProcessingStages ++ Array(lr))

In [ ]:
// Fit the pipeline to training documents.
val model = pipeline.fit(training)

### fitting model to test set

In [ ]:
val prediction = model.transform(test)

### model evaluation

In [ ]:
getConfusionMatrix(prediction)